# CORE - A Cell-Level Coarse-to-Fine Image Registration Engine for Multi-stain Image Alignment

This notebook demonstrates the coarse level Whole Slide Image (WSI) registration using CORE method.

## Overview
- **Coarse Registration**: Initial coarse registration using CORE

## 1. Setup and Imports

In [1]:
import sys
import os

# Add project root to PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # adjust if needed
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [ ]:
# Enable inline plotting and auto-reload for development
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import all necessary modules
import SimpleITK as sitk 
from core.utils.imports import *
from core.config import *
from core.preprocessing.preprocessing import *
from core.preprocessing.padding import *
from core.registration.registration import *
from core.registration.nonrigid import *
from core.evaluation.evaluation import *
from core.visualization.visualization import *
from core.preprocessing.nuclei_analysis import *

# Setup Bokeh for notebook output
setup_bokeh_notebook()

print("✅ All modules imported successfully!")
print(f"Source WSI: {SOURCE_WSI_PATH}")
print(f"Target WSI: {TARGET_WSI_PATH}")

## 2. Configuration Check

Verify that all file paths are correct and files exist.

## 3. Load and Preprocess WSI Images

In [ ]:
import os

# Check if files exist
files_to_check = [
    SOURCE_WSI_PATH,
    TARGET_WSI_PATH,
    FIXED_POINTS_PATH,
    MOVING_POINTS_PATH
]

print("File existence check:")
for file_path in files_to_check:
    exists = os.path.exists(file_path)
    status = "✅" if exists else "❌"
    print(f"{status} {file_path}")

# Display current parameters
print("\nCurrent Parameters:")
print(f"- Preprocessing Resolution: {PREPROCESSING_RESOLUTION}")
print(f"- Registration Resolution: {REGISTRATION_RESOLUTION}")
print(f"- Patch Size: {PATCH_SIZE}")
print(f"- Fixed Threshold: {FIXED_THRESHOLD}")
print(f"- Moving Threshold: {MOVING_THRESHOLD}")
print(f"- Min Nuclei Area: {MIN_NUCLEI_AREA}")

In [ ]:
# Load WSI images
print("Loading WSI images...")
source_wsi, target_wsi, source, target = load_wsi_images(
    SOURCE_WSI_PATH, TARGET_WSI_PATH, PREPROCESSING_RESOLUTION
)

print(f"\nLoaded images:")
print(f"Source shape: {source.shape}")
print(f"Target shape: {target.shape}")

In [ ]:
import matplotlib.cm as cm
# Preprocess images
print("Preprocessing images...")
source_prep,target_prep, padding_params=pad_images(source, target)
# Extract tissue masks
print("Extracting tissue masks...")
# Set artefacts to true if your slide contains control tissue as well
source_mask, target_mask = extract_tissue_masks(source_prep, target_prep, artefacts=False)
print("✅ Preprocessing completed!")

## 4. Visualize Original Images and Tissue Masks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# set slide to mif if visualizing multiplex-if dapi
if slide=='mif'
    # Create a blank RGB image
    colored = np.zeros((*target_prep.shape, 3), dtype=np.uint8)

    # Apply colormap only on the masked region
    masked_region = target_prep.copy()
    masked_region[target_mask == 0] = 0  # zero out regions outside the mask

    colored_masked = cv2.applyColorMap(masked_region, cv2.COLORMAP_JET)
    colored_masked = cv2.cvtColor(colored_masked, cv2.COLOR_BGR2RGB)
else:
    colored_masked=taget
# Display side by side
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0, 0].imshow(source_prep)
axes[0, 0].set_title('Source Image (Moving)')
axes[0, 0].axis('off')

axes[0, 1].imshow(colored_masked)
axes[0, 1].set_title('Target Image (Masked & Colored)')
axes[0, 1].axis('off')

axes[1, 0].imshow(source_mask, cmap='gray')
axes[1, 0].set_title('Source Tissue Mask')
axes[1, 0].axis('off')

axes[1, 1].imshow(target_mask, cmap='gray')
axes[1, 1].set_title('Target Tissue Mask')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()


## 5. Perform Rigid Registration

In [ ]:
# Perform rigid registration
print("Performing rigid registration...")
moving_img_transformed, final_transform = perform_rigid_registration(
    source_prep, target_prep, source_mask, target_mask
)
# Visualize transformed image
plt.figure(figsize=(12, 8))
plt.imshow(moving_img_transformed)
plt.title('Transformed Source Image')
plt.axis('off')
plt.show()


In [ ]:
r_x, r_y = util.matrix_df(source_prep,np.linalg.inv(final_transform))
rigid_field = np.stack(( r_x, r_y), axis=-1)
sitk_image = sitk.GetImageFromArray(rigid_field)
# sitk.WriteImage(sitk_image, './anhir_rigid.mha')

In [ ]:

visualize_overlays(target_prep,source_prep, moving_img_transformed)

## 6. Non Rigid Registration

In [ ]:
displacement_field,warped_source= elastic_image_registration(
   moving_img_transformed,target_prep)
print("non rigid displacement field",displacement_field.shape)
create_deform(source_prep, final_transform, displacement_field, output_path="")

## 7. Visualization

In [ ]:
%%bash 
tiatoolbox visualize --slides "path/to/source/slide/folder" --overlays "path/to/target/slide/and/deformation/field/folder"